In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('review video').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/20 18:16:14 WARN Utils: Your hostname, Pulastyas-Mac-mini.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface en1)
26/01/20 18:16:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 18:16:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
user_flags = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/user_flags.csv')

user_flags.show()

+--------------+-------------+-----------+-------+
|user_firstname|user_lastname|   video_id|flag_id|
+--------------+-------------+-----------+-------+
|       Richard|       Hasson|y6120QOlsfU| 0cazx3|
|          Mark|          May|Ct6BUPvE2sM| 1cn76u|
|          Gina|       Korman|dQw4w9WgXcQ| 1i43zk|
|          Mark|          May|Ct6BUPvE2sM| 1n0vef|
|          Mark|          May|jNQXAC9IVRw| 1sv6ib|
|          Gina|       Korman|dQw4w9WgXcQ| 20xekb|
|          Mark|          May|5qap5aO4i9A| 4cvwuv|
|        Daniel|         Bell|5qap5aO4i9A| 4sd6dv|
|       Richard|       Hasson|y6120QOlsfU| 6jjkvn|
|       Pauline|        Wilks|jNQXAC9IVRw| 7ks264|
|      Courtney|         NULL|dQw4w9WgXcQ|   NULL|
|         Helen|        Hearn|dQw4w9WgXcQ| 8946nx|
|          Mark|      Johnson|y6120QOlsfU| 8wwg0l|
|       Richard|       Hasson|dQw4w9WgXcQ| arydfd|
|          Gina|       Korman|       NULL|   NULL|
|          Mark|      Johnson|y6120QOlsfU| bl40qw|
|       Richard|       Hasson|d

In [4]:
flag_review = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/flag_review.csv')

flag_review.show()

+-------+--------------+-------------+----------------+
|flag_id|reviewed_by_yt|reviewed_date|reviewed_outcome|
+-------+--------------+-------------+----------------+
| 0cazx3|         false|         NULL|            NULL|
| 1cn76u|          true|   2022-03-15|         REMOVED|
| 1i43zk|          true|   2022-03-15|         REMOVED|
| 1n0vef|          true|   2022-03-15|         REMOVED|
| 1sv6ib|          true|   2022-03-15|        APPROVED|
| 20xekb|          true|   2022-03-17|         REMOVED|
| 4cvwuv|          true|   2022-03-15|        APPROVED|
| 4l1tk7|         false|         NULL|            NULL|
| 4sd6dv|          true|   2022-03-14|         REMOVED|
| 6jjkvn|          true|   2022-03-16|        APPROVED|
| 7ks264|          true|   2022-03-15|        APPROVED|
| 8946nx|         false|         NULL|            NULL|
| 8wwg0l|         false|         NULL|            NULL|
| arydfd|          true|   2022-03-15|        APPROVED|
| bl40qw|          true|   2022-03-16|         R

In [9]:
result = user_flags.join(flag_review, on='flag_id')\
    .select(F.col('video_id'), F.col('reviewed_by_yt'))\
    .filter(F.col('reviewed_by_yt') == True)\
    .groupBy(F.col('video_id')).agg(F.count('*').alias('reviewed_by_yt'))

result.show()

+-----------+--------------+
|   video_id|reviewed_by_yt|
+-----------+--------------+
|y6120QOlsfU|             3|
|5qap5aO4i9A|             3|
|Ct6BUPvE2sM|             2|
|dQw4w9WgXcQ|             5|
|jNQXAC9IVRw|             4|
+-----------+--------------+



26/01/20 18:58:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1044916 ms exceeds timeout 120000 ms
26/01/20 18:58:57 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/20 18:58:59 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$